In [13]:
import pandas as pd
import numpy as np
import ast

In [3]:
dset= pd.read_csv('dhaka_div.csv')

In [4]:
dset.columns

Index(['osm_type', 'name', 'category', 'type', 'lat', 'lon', 'tags'], dtype='object')

In [12]:
dset.drop('id',axis = 1, inplace=True)
dset.head()

,osm_type,name,category,type,lat,lon,tags
0,node,German Club - Rest House Society,amenity,pub,23.793577,90.419754,"{'addr:city': 'Dhaka', 'addr:country': 'BD', '..."
1,node,American Club - AEEA,amenity,pub,23.802524,90.410248,"{'amenity': 'pub', 'name': 'American Club - AE..."
2,node,Netherland Recreation Center (Dutch Club),amenity,pub,23.803080,90.413906,"{'amenity': 'pub', 'name': 'Netherland Recreat..."
3,node,NaN,amenity,ferry_terminal,23.894661,90.571004,"{'amenity': 'ferry_terminal', 'ferry': 'yes', ..."
4,node,NaN,amenity,ferry_terminal,23.621013,90.452398,"{'amenity': 'ferry_terminal', 'ferry': 'yes', ..."


In [21]:
# print(type(dset['tags'][12]))
# x = ast.literal_eval(dset['tags'][12])
# dset['tags'][12]=x
# print(type(dset['tags'][12]))
# currently all data is in str form so it needs to convern into dict
# dset['tags'] = dset['tags'].apply(ast.literal_eval)
def safe_eval(v):
    if isinstance(v, str):
        try:
            return ast.literal_eval(v)
        except:
            return v   
    return v  
dset['tags'] = dset['tags'].apply(safe_eval)

In [55]:
# dset['type'].isna()[8] #np.True_
# pd.isna(dset.loc[8, 'type']) #True
dset['type'].isna().sum()

np.int64(5312)

In [72]:
for i in range(len(dset)):
    if pd.isna(dset.loc[i, 'type']) :
        x= dset['tags'][i]
        if 'railway' in x:
            print(x)

{'railway': 'level_crossing'}
{'crossing:barrier': 'double_half', 'railway': 'level_crossing'}
{'railway': 'level_crossing'}
{'railway': 'level_crossing'}
{'crossing:barrier': 'full', 'name': 'মালিবাগ রেল গেট', 'name:bn': 'মালিবাগ রেল গেট', 'name:en': 'Malibag Rail Gate', 'railway': 'level_crossing', 'source': 'local lnowledge'}
{'railway': 'signal'}
{'railway': 'level_crossing'}
{'railway': 'level_crossing'}
{'railway': 'level_crossing'}
{'railway': 'level_crossing'}
{'railway': 'level_crossing'}
{'railway': 'crossing'}
{'railway': 'switch'}
{'railway': 'level_crossing'}
{'crossing:barrier': 'double_half', 'railway': 'level_crossing'}
{'railway': 'level_crossing'}
{'railway': 'level_crossing'}
{'railway': 'level_crossing'}
{'railway': 'level_crossing'}
{'railway': 'level_crossing'}
{'railway': 'level_crossing'}
{'name': 'চাষাড়া', 'name:en': 'Chasara', 'operator': 'বাংলাদেশ রেলওয়ে', 'operator:en': 'Bangladesh Railway', 'railway': 'station'}
{'railway': 'crossing'}
{'designation': 'Rail

In [66]:
t=0
for i in range(len(dset)):
        x= dset['tags'][i]
        if 'railway' in x:
            t=t+1
print(t)

1223


In [68]:
for i in range(len(dset)):
    if (not pd.isna(dset.loc[i, 'type'])) and ('railway' in dset.loc[i, 'tags']):
        print(i)


9230


In [71]:
dset.loc[9230, ['tags', 'type']]

tags    {'amenity': 'vending_machine', 'name': 'কমলাপু...
type                                      vending_machine
Name: 9230, dtype: object